# Iris CLASSIFIER
### Let's build a quantum variational algorithm to classify the Iris flowers

In [1]:
# First, we import the libaries
import quforge.quforge as qf
import random

In [2]:
# Load the dataset
from sklearn import datasets

iris = datasets.load_iris()
data = iris['data']
labels = iris['target']

ids = random.sample(range(len(data)), len(data))
ids_train = ids[0:int(0.8*len(data))]
ids_test = ids[int(0.8*len(data)):len(data)]

In [3]:
# Build the model

class Circuit(qf.Module):
    def __init__(self, dim, wires, device):
        super(Circuit, self).__init__()

        self.dim = dim
        self.init = qf.H(dim=dim, 
                         index=range(wires), 
                         device=device)
        self.encoder = qf.RZ(dim=dim, 
                             index=range(wires), 
                             device=device)

        self.circuit = qf.Circuit(dim=dim, 
                                  wires=wires, 
                                  device=device)
        self.circuit.RX(j=0, k=1, index=range(wires))
        self.circuit.RX(j=1, k=2, index=range(wires))
        self.circuit.RX(j=0, k=2, index=range(wires))
        self.circuit.RY(j=0, k=1, index=range(wires))
        self.circuit.RY(j=1, k=2, index=range(wires))
        self.circuit.RY(j=0, k=2, index=range(wires))
        self.circuit.RZ(j=1, index=range(wires))
        self.circuit.RZ(j=2, index=range(wires))
        self.circuit.CNOT(index=[0,1])
        self.circuit.CNOT(index=[0,2])
        self.circuit.CNOT(index=[0,3])
        self.circuit.RX(j=0, k=1, index=range(wires))
        self.circuit.RX(j=1, k=2, index=range(wires))
        self.circuit.RX(j=0, k=2, index=range(wires))
        self.circuit.RY(j=0, k=1, index=range(wires))
        self.circuit.RY(j=1, k=2, index=range(wires))
        self.circuit.RY(j=0, k=2, index=range(wires))
        self.circuit.RZ(j=1, index=range(wires))
        self.circuit.RZ(j=2, index=range(wires))

        self.initial_state = qf.State('0-0-0-0', 
                                      dim=dim, 
                                      device=device)

    def forward(self, x):
        y = self.init(self.initial_state)
        y = self.encoder(y, param=x)
        y = self.circuit(y)

        return y

In [4]:
# Instantiate model and optimizer
dim = 3 #dimension of the qudit
wires = 4 #number of qudits
device = 'cuda'

model = Circuit(dim=dim, wires=wires, device=device)
optimizer = qf.optim.Adam(model.parameters(), lr=0.001)

In [5]:
# Prepare the target states
targets = [qf.State('1-0-0', dim=dim, device=device).flatten(), qf.State('0-1-0', dim=dim, device=device).flatten(), qf.State('0-0-1', dim=dim, device=device).flatten()]
targets_arg = [qf.argmax(abs(targets[0])), qf.argmax(abs(targets[1])), qf.argmax(abs(targets[2]))]

In [7]:
# Optimize the model
for epoch in range(2):
    acc_train = 0.0
    acc_test = 0.0
    
    for k in ids_train:
        x = data[k]
        label = labels[k]
        output = model(x)
        
        _, m = qf.measure(output, index=[0, 1, 2], dim=dim, wires=wires)
        
        loss = qf.mean(abs(targets[label] - m))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        predict = qf.argmax(m)
        if predict == targets_arg[label]:
            acc_train += 1

    for k in ids_test:
        x = data[k]
        label = labels[k]
        output = model(x)
        
        _, m = qf.measure(output, index=[0, 1, 2], dim=dim, wires=wires)

        predict = qf.argmax(m)
        if predict == targets_arg[label]:
            acc_test += 1
    
    acc_train = acc_train/len(ids_train)
    acc_test = acc_test/len(ids_test)

    print(epoch, acc_train, acc_test)

0 0.475 0.5
1 0.575 0.5
